# Data Modelling

In [62]:
from pyspark.sql.session import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from helpers.helper_functions import translate_to_file_string
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics

inputFile = translate_to_file_string("./data/Data_Preparation_Result.csv")

## Create Spark Session

In [36]:
#create a SparkSession
spark = (SparkSession
       .builder
       .appName("DataModelling")
       .getOrCreate())
# create a DataFrame using an ifered Schema 
df = spark.read.option("header", "true") \
       .option("inferSchema", "true") \
       .option("delimiter", ";") \
       .csv(inputFile)   
print(df.printSchema())

root
 |-- Bundesland: string (nullable = true)
 |-- BundeslandIndex: integer (nullable = true)
 |-- Landkreis: string (nullable = true)
 |-- LandkreisIndex: integer (nullable = true)
 |-- Altersgruppe: string (nullable = true)
 |-- AltersgruppeIndex: double (nullable = true)
 |-- Geschlecht: string (nullable = true)
 |-- GeschlechtIndex: double (nullable = true)
 |-- FallStatus: string (nullable = true)
 |-- FallStatusIndex: double (nullable = true)
 |-- Falldatum: string (nullable = true)

None


## Vorbereitung der Daten

### Filtern der Datensätze
Für das Training dieses Modells ist es sinnvoll nur die Fälle zu betrachten, bei den der Ausgang der Corona-Erkrankung bereits bekannt ist ("GENESEN" oder "GESTORBEN"). Daher werden die Fälle mit noch erkrankten Personen herausgefiltert. Ebenfalls muss der FallStatusIndex neu vergeben werden, damit dieses Feature nur noch die Werte 0 oder 1 enthält.Dies erfolgt im nachfolgenden Abschnitt.

In [37]:
dfNeu = df.filter(df.FallStatus != "NICHTEINGETRETEN").drop("FallStatusIndex")

### FallStatusIndex

In [38]:
# Wollten eigentlich den Indexer mitgeben in die Pipeline. Dies führt aber zum Fehler, dass er das Label nicht kennt.
indexer = StringIndexer(inputCol="FallStatus", outputCol="FallStatusIndex")

### Aufbau des Feature-Vectors

In [39]:
assembler =  VectorAssembler(outputCol="features", inputCols=["GeschlechtIndex","AltersgruppeIndex","LandkreisIndex"])

### Splitten in Trainings und Testdaten

In [40]:
splits = dfNeu.randomSplit([0.8, 0.2 ], 345678)
trainingVector = splits[0]
testVector = splits[1]

trainingVector.limit(10).show()

+-----------------+---------------+-----------+--------------+------------+-----------------+----------+---------------+----------+----------+
|       Bundesland|BundeslandIndex|  Landkreis|LandkreisIndex|Altersgruppe|AltersgruppeIndex|Geschlecht|GeschlechtIndex|FallStatus| Falldatum|
+-----------------+---------------+-----------+--------------+------------+-----------------+----------+---------------+----------+----------+
|Baden-Württemberg|              8|LK Biberach|          8426|     A35-A59|              0.0|         M|            1.0| GESTORBEN|2021-03-29|
|Baden-Württemberg|              8|LK Biberach|          8426|     A35-A59|              0.0|         M|            1.0| GESTORBEN|2021-04-11|
|Baden-Württemberg|              8|LK Biberach|          8426|     A35-A59|              0.0|         M|            1.0| GESTORBEN|2021-04-11|
|Baden-Württemberg|              8|LK Biberach|          8426|     A35-A59|              0.0|         M|            1.0| GESTORBEN|2021-04-24|

## Modellierung
### Support Vector Machine

In [41]:
lsvc = LinearSVC(featuresCol="features", labelCol="FallStatusIndex")

### Pipeline

In [42]:
pipeline = Pipeline(stages=[indexer,assembler, lsvc])

### Parametertuning
Eine wichtige Aufgabe beim Machine Learning ist die Auswahl des geeigneten Modells bzw. die passenden Paramter für ein Modell herauszufinden. Letzteres wird auch Parametertuning genannt. Die in Pyspark enthaltene MLLib bietet speziell hierfür ein entsprechende Tooling. Und zwar kann ein CrossValidator bzw. ein TrainValidationSplit verwendet werden. Voraussetzung sind ein Estimator (ein Modell oder eine Pipeline), ein Paramter-Grid und eine Evaluator. Dies ist auch im Zusammenhang mit dem Thema Cross-Validation zu sehen. (Apache Spark 2020a)

In [43]:
paramGrid = ParamGridBuilder()\
    .addGrid(lsvc.regParam, [1, 0.1]) \
    .addGrid(lsvc.maxIter, [10,50])\
    .build()

### Cross-Validation
Wie eben erwähnt, benötigt der Cross-Validator einen Evaluator. Letzterer ist zu wählen, abhängig von dem jeweilligen Modell und Anwendungsfall. (Apache Spark 2020a) In diesem Fall wird ein BinaryClassificationEvaluator angewendet. Dieser eignet sich besonders für binäre Werte. (Apache Spark 2021a) Da Geschlecht, in diesem Fall, der FallStatus 0 oder 1 annehmen kann, bietet er sich hier besonders an.

In [44]:
# Definition des Evaluators
evaluator = BinaryClassificationEvaluator(labelCol="FallStatusIndex",rawPredictionCol="rawPrediction", metricName="areaUnderROC")

In [45]:
# Definition des Cross-Validators 
# num-Folds gibt an in wie viele Datensatz-Paare die Datensätze aufgeteilt werden.
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3,
                          parallelism=3)

#### Durchführung

In [46]:
# Anpassung des Modells und Auswahl der besten Parameter
cvModel = crossval.fit(trainingVector)

### Testen des Modells

In [47]:
predictions = cvModel.transform(testVector)
predictions.show()

+-----------------+---------------+----------------+--------------+------------+-----------------+----------+---------------+----------+----------+---------------+----------------+--------------------+----------+
|       Bundesland|BundeslandIndex|       Landkreis|LandkreisIndex|Altersgruppe|AltersgruppeIndex|Geschlecht|GeschlechtIndex|FallStatus| Falldatum|FallStatusIndex|        features|       rawPrediction|prediction|
+-----------------+---------------+----------------+--------------+------------+-----------------+----------+---------------+----------+----------+---------------+----------------+--------------------+----------+
|Baden-Württemberg|              8|     LK Biberach|          8426|     A60-A79|              2.0|         M|            1.0| GESTORBEN|2020-04-14|            1.0|[1.0,2.0,8426.0]|[1.00027182582988...|       0.0|
|Baden-Württemberg|              8|     LK Biberach|          8426|     A60-A79|              2.0|         M|            1.0| GESTORBEN|2020-12-08| 

In [77]:
predictions.groupBy("prediction").count().show()


+----------+------+
|prediction| count|
+----------+------+
|       0.0|711068|
+----------+------+



In [ ]:
predictions.groupBy("prediction").count().show()


## Modell - Evaluation

### Area Under Roc

In [54]:
accuracy = evaluator.evaluate(predictions)
print("Test Error",(1.0 - accuracy))

Test Error 0.8426292151881036


### BinaryClassificationMetrics

Bei dem untersuchten Label (FallStatus mit den Ausprägungen Verstorben und Genesen) handelt es sich um ein einen BinaryClasificator. Er kann die Werte 0 und 1 annehmen. Für die Modellevaluation sind daher die BinaryClassificationMetrics zu verwenden. (Apache Spark 2021i)

In [65]:
predictionAndLabels = predictions.select("prediction", "FallStatusIndex").rdd.map(lambda p: [p[0], float(p[1])]) # Map to RDD prediction|label

In [66]:
# Instanzieire das BinaryClassificationMetrics-Objekt
metrics = BinaryClassificationMetrics(predictionAndLabels)

# Fläche unter der Precision-recall Curve

print("Area under PR = %s" % metrics.areaUnderPR)

# Fläche unter der ROC curve
print("Area under ROC = %s" % metrics.areaUnderROC)

Area under PR = 0.024765563912312185
Area under ROC = 0.5


### Multiclass classification Metrics
In den meißten Fällen können auch Multiclass Classification Metrics bei Binary Classifaction Problemen angewandt werden.

In [67]:
# Instantiate metrics object
metrics = MulticlassMetrics(predictionAndLabels)


In [74]:
# Overall statistics
precision = metrics.precision(1.0)
recall = metrics.recall(1.0)
f1Score = metrics.fMeasure(1.0)
print("Summary Stats")
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1 Score = %s" % f1Score)


labels = predictions.select("FallStatusIndex").rdd.map(lambda lp: lp.FallStatusIndex).distinct().collect()
for label in sorted(labels):
    print("Class %s precision = %s" % (label, metrics.precision(label)))
    print("Class %s recall = %s" % (label, metrics.recall(label)))
    print("Class %s F1 Measure = %s" % (label, metrics.fMeasure(label, beta=1.0)))

# Weighted stats
print("Weighted recall = %s" % metrics.weightedRecall)
print("Weighted precision = %s" % metrics.weightedPrecision)
print("Weighted F(1) Score = %s" % metrics.weightedFMeasure())
print("Weighted F(0.5) Score = %s" % metrics.weightedFMeasure(beta=0.5))
print("Weighted false positive rate = %s" % metrics.weightedFalsePositiveRate)

Summary Stats
Precision = 0.0
Recall = 0.0
F1 Score = 0.0
Class 0.0 precision = 0.9752344360876878
Class 0.0 recall = 1.0
Class 0.0 F1 Measure = 0.9874619622563057
Class 1.0 precision = 0.0
Class 1.0 recall = 0.0
Class 1.0 F1 Measure = 0.0
Weighted recall = 0.9752344360876878
Weighted precision = 0.9510822053312705
Weighted F(1) Score = 0.96300690991907
Weighted F(0.5) Score = 0.9558164721170012
Weighted false positive rate = 0.9752344360876878
